Happy Transformer: https://github.com/EricFillion/happy-transformer

In [ ]:
!pip install happytransformer

In [2]:
import pandas as pd
import psycopg2
from happytransformer import HappyTextClassification
from concurrent.futures import ThreadPoolExecutor

# Database connection details
username = "tsdbadmin"
password = "nzk71o1l4kkl7hg5"
host = "are42sp6i4.jb5mfu0e69.tsdb.cloud.timescale.com"
port = "33785"
db_name = "tsdb"


# Sentiment Analysis Setup
happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)
tokenizer = happy_tc.tokenizer
MAX_TOKENS = 510


# Function to classify sentiment
def classify_sentiment(text):
    tokens = tokenizer.tokenize(text)
    if len(tokens) > MAX_TOKENS:
        tokens = tokens[:MAX_TOKENS]
        text = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens))
    return happy_tc.classify_text(text).label


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
batch_size = 2000

def process_batch(batch_id, conn_details):
    conn = psycopg2.connect(**conn_details, sslmode='require')
    cursor = conn.cursor()

    query = f"SELECT id, \"reviewText\" FROM \"Review\" WHERE sentiment = 'NULL' ORDER BY id OFFSET {batch_id * batch_size} LIMIT {batch_size}"
    df = pd.read_sql(query, conn)

    if df.empty:
        return

    df['sentiment'] = df['reviewText'].apply(classify_sentiment)

    update_data = df[['sentiment', 'id']].values.tolist()
    update_query = "UPDATE \"Review\" SET sentiment = %s WHERE id = %s"
    cursor.executemany(update_query, update_data)
    conn.commit()

    cursor.close()
    conn.close()

# Database connection details
conn_details = {
    "dbname": db_name,
    "user": username,
    "password": password,
    "host": host,
    "port": port
}

# Number of threads and batches to process
num_threads = 12  # Adjust based on your system's capability and database server's capacity
num_batches = 210  # Total number of batches to process

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_batch, i, conn_details) for i in range(num_batches)]

    # Optionally, you can handle the results or exceptions from each future
    for future in futures:
        try:
            future.result()  # This will re-raise any exception raised in the thread
        except Exception as e:
            print(f"Error processing batch: {e}")


<ipython-input-3-284eea26efa8>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors
<ipython-input-3-284eea26efa8>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
<ipython-input-3-284eea26efa8>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
<ipython-input-3-

In [ ]:
# # Database Connection
# conn = psycopg2.connect(dbname=db_name, user=username, password=password, host=host, port=port, sslmode='require')
# cursor = conn.cursor()

# # Iterate over the database in batches
# batch_size = 6000

# while True:
#     query = f"SELECT id, \"reviewText\" FROM \"Review\" WHERE sentiment = 'NULL' ORDER BY id LIMIT {batch_size}"
#     df = pd.read_sql(query, conn)

#     if df.empty:
#         break


#     # Apply sentiment classification
#     df['sentiment'] = df['reviewText'].apply(classify_sentiment)

#     # Prepare data for bulk update
#     update_data = df[['sentiment', 'id']].values.tolist()

#     # Bulk update the original table
#     update_query = "UPDATE \"Review\" SET sentiment = %s WHERE id = %s"
#     cursor.executemany(update_query, update_data)
#     conn.commit()

# # Close the cursor and connection
# cursor.close()
# conn.close()